In [ ]:
from math import sqrt as sqrt
from qiskit import IBMQ, QuantumCircuit, transpile, execute
from qiskit.providers.aer import AerSimulator
from qiskit.tools.monitor import job_monitor

def loginQComputer(api_token) :
    IBMQ.save_account(api_token)
    IBMQ.load_account()
    
def getQComputerService() :
    provider = IBMQ.get_provider('ibm-q')
    return provider.get_backend('ibmq_manila')

def showQState(statistic) :
    all_state = sorted(statistic.keys())
    total_count = sum(statistic.values())
    print("\nQuantum State (Total execution time: {})".format(total_count))
    for i in all_state :
        print("|{}〉: {:.2f}%".format(i, statistic[i]/total_count*100))

# log in
apiToken = 'cff46de382c63d792161c08a2922a995180e0beb2cd3a0d14ff73b0e04ed760c0583204256c4968c8d7c28ed5a64447a8698cff9bbc5e6d0d1254d0115a0c588'
loginQComputer(apiToken)

# create a quantum circuit
qCircuit = QuantumCircuit(1, 1)

# initialize the quantum circuit
qState = [1/3+2/3j, sqrt(3)/3+1/3j]
qCircuit.initialize(qState, 0)

# show the quantum circuit
qBits = [i for i in range(1)]
cBits = [i for i in range(1)]
qCircuit.measure(qBits, cBits)
print(qCircuit)

# test in Simulator
"""qSimulator = AerSimulator()
qCode = transpile(qCircuit, qSimulator)
execution = execute(qCode, backend=qSimulator, shots=1000)"""

# run in real quantum computer
qComputer = getQComputerService()
qCode = transpile(qCircuit, qComputer)
execution = execute(qCode, backend=qComputer, shots=1000)

# monitor the job status
print()
job_monitor(execution)

# show the result
result = execution.result()
statistic = result.get_counts(qCircuit)
showQState(statistic)